In [1]:
import pandas as pd
import numpy as np
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe
import json # library to handle JSON files
import requests,json
import folium
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn import preprocessing


In [19]:
riyadh_data=pd.read_csv('riyadh_cleaned.csv')
riyadh_data.head()

,Unnamed: 0,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category,Verified,Likes,Rate
0,0,malga,24.800735,46.600703,Tamimi Markets (أسواق التميمي),24.801425,46.602780,Supermarket,False,1366,9.4
1,1,malga,24.800735,46.600703,Starbucks (ستاربكس),24.801419,46.602797,Coffee Shop,False,450,9.0
2,2,malga,24.800735,46.600703,Juice Time (وقت العصير),24.802362,46.603178,Juice Bar,False,73,9.2
3,3,malga,24.800735,46.600703,Dunkin' Donuts (دانكن دونتس),24.801518,46.603399,Donut Shop,False,158,8.9
4,4,malga,24.800735,46.600703,Anoosh (أنوش),24.802679,46.603831,Dessert Shop,False,249,9.1


In [20]:
CLIENT_ID = 'SBRUPO0JNQJ42WN0Y45KJ3MHWUHPOEHP4TAKDLTWSK5IKKT5' # your Foursquare ID
CLIENT_SECRET = 'GANLLZLYVEU1K5VT0FSKECGQFIM3VDZN0JMXKT4MZCHKHTZS' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

In [11]:
lat=''
lng=''
url = 'https://api.foursquare.com/v2/venues/categories?&client_id={}&client_secret={}&v={}'.format(CLIENT_ID,
                                                                                                   CLIENT_SECRET,
                                                                                                   VERSION)
categories=requests.get(url).json()

In [12]:
cats=[]
for cat in categories['response']['categories']:
    if cat['name']=='Food':
        for f_cat in cat['categories']:
            cats.append([f_cat['id'],f_cat['name']])

In [66]:
cat_ids_lst=[lst[0] for lst in cats ]
cat_ids_lst[0:7].append(cat_ids_lst[9:])


### iterate on neighborhood to get venues by categories id

In [22]:
neighborhoods=riyadh_data[riyadh_data[['Neighborhood Latitude','Neighborhood Longitude']].duplicated()==False]
neighborhoods

,Unnamed: 0,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category,Verified,Likes,Rate
0,0,malga,24.800735,46.600703,Tamimi Markets (أسواق التميمي),24.801425,46.602780,Supermarket,False,1366,9.4
100,100,sahafah,24.797262,46.640168,FROZ (فروز),24.797950,46.641953,Frozen Yogurt Shop,False,390,9.0
115,116,Aqeeq,24.779879,46.626809,Burj Rafal Hotel (فندق برج رافال),24.792501,46.632336,Hotel,True,3461,9.2
140,149,Yasmin,24.822791,46.639762,أسواق ومخابز جميرا الامارات,24.808072,46.638413,Grocery Store,False,52,9.1
216,249,Hittin,24.753889,46.600833,O100,24.758679,46.605999,Perfume Shop,False,58,9.3
338,372,Nargis,24.834828,46.674293,Starbucks (ستاربكس),24.822058,46.649758,Coffee Shop,False,659,9.3
425,478,Mursalat,24.746928,46.690852,ج Gym,24.739044,46.691740,Gym / Fitness Center,False,63,9.1
610,664,muruj,24.757762,46.663673,Juice Time (وقت العصير),24.759272,46.664003,Juice Bar,False,255,8.9


In [ ]:
8*92

In [29]:
#get venues for each Neighborhood
#parent_df=pd.DataFrame(columns=child_df.columns)
for neigh,lat,lng in zip(neighborhoods['Neighborhood'],neighborhoods['Neighborhood Latitude'],neighborhoods['Neighborhood Longitude']):
    for cat_id in cat_ids_lst:
        url='https://api.foursquare.com/v2/venues/search?categoryId={}\
        &client_id={}&client_secret={}&v={}&ll={},{}&limit=500&raduis=3000'.format(cat_id,CLIENT_ID,CLIENT_SECRET,VERSION,lat,lng)
        res=None
        try:
            res=requests.get(url).json()
            child_df=json_normalize(res['response']['venues'])
        except:
            print(res)
            #break
        child_df['Neighborhood']=neigh
        parent_df=pd.concat([parent_df,child_df],axis=0)

        #break
#res

G:\Users\SAAD\anaconda3\lib\site-packages\ipykernel_launcher.py:11: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  # This is added back by InteractiveShellApp.init_path()


{'meta': {'code': 500, 'errorType': 'server_error', 'errorDetail': 'Foursquare servers are experiencing problems. Please retry and check status.foursquare.com for updates.'}, 'response': {}}
{'meta': {'code': 500, 'errorType': 'server_error', 'errorDetail': 'Foursquare servers are experiencing problems. Please retry and check status.foursquare.com for updates.'}, 'response': {}}
None
None
None
None
None
None


In [28]:
parent_df=pd.DataFrame(columns=child_df.columns)
parent_df

,id,name,categories,referralId,hasPerk,location.lat,location.lng,location.labeledLatLngs,location.distance,location.cc,location.neighborhood,location.country,location.formattedAddress,location.postalCode,location.city,location.state,location.address,location.crossStreet,Neighborhood


In [84]:
#len(parent_df.columns)
parent_df.shape

(4155, 20)

In [68]:
parent_df.head()

,id,name,categories,referralId,hasPerk,location.lat,location.lng,location.labeledLatLngs,location.distance,location.cc,location.neighborhood,location.country,location.formattedAddress,location.postalCode,location.city,location.state,location.address,location.crossStreet,Neighborhood,venuePage.id
0,567e7951498e0671db8a7314,مطعم سمرقندي,Afghan Restaurant,v-1593668315,False,24.580698,46.602932,"[{'label': 'display', 'lat': 24.580698, 'lng':...",24495,SA,Al-Uraija,المملكة العربية السعودية,[المملكة العربية السعودية],NaN,NaN,NaN,NaN,NaN,malga,NaN
1,58709bd92520ae35b5fe1dd3,السرير الي اكل فيه,Afghan Restaurant,v-1593668315,False,24.590693,46.638974,"[{'label': 'display', 'lat': 24.590693, 'lng':...",23699,SA,NaN,المملكة العربية السعودية,[المملكة العربية السعودية],NaN,NaN,NaN,NaN,NaN,malga,NaN
2,5ee20d8b943a3857fcfb7427,NAZ Restaurant مطعم ناز,Afghan Restaurant,v-1593668315,False,24.863680,46.642653,"[{'label': 'display', 'lat': 24.86368, 'lng': ...",8188,SA,NaN,المملكة العربية السعودية,"[العارض الرياض (طريق ابو بكر الصديق), الرياض 1...",13335,الرياض,منطقة الرياض‎,العارض الرياض,طريق ابو بكر الصديق,malga,NaN
3,5dbe3f380c7f49000884de69,الفوال اللي يعرف الطلب بدون مانقوله,Afghan Restaurant,v-1593668315,False,24.721318,46.849321,"[{'label': 'display', 'lat': 24.721318, 'lng':...",26641,SA,NaN,المملكة العربية السعودية,"[الرياض 14245, المملكة العربية السعودية]",14245,الرياض,منطقة الرياض‎,NaN,NaN,malga,NaN
4,5c93dcc4dab4b1002c3f4def,"اطلق بخاري بالمونسية "" اطلب على الفحم و رز احم...",Afghan Restaurant,v-1593668315,False,24.821028,46.764303,"[{'label': 'display', 'lat': 24.821028, 'lng':...",16684,SA,NaN,المملكة العربية السعودية,"[الرياض 13246, المملكة العربية السعودية]",13246,الرياض,منطقة الرياض‎,NaN,NaN,malga,NaN


In [69]:
# extract categroy from json
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [64]:
parent_df['categories']=parent_df.apply(get_category_type,axis=1)

In [ ]:
parent_df['location.state'].unique()

In [83]:
parent_df[parent_df.id.duplicated()].head()

G:\Users\SAAD\anaconda3\lib\site-packages\ipykernel_launcher.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  """Entry point for launching an IPython kernel.


ValueError: cannot reindex from a duplicate axis

In [86]:
#remove duplicates
parent_df.drop

TypeError: unhashable type: 'list'

In [ ]:
parent_df.reset_index(drop=True,inplace=True)

In [ ]:
#drop data frame values
parent_df=parent_df.drop(list(np.arange(parent_df.index.start,parent_df.index.stop)),axis=0)
parent_df